<a href="https://colab.research.google.com/github/Machine-Learning-Tokyo/DL-workshop-series/blob/master/Simple_GAN_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple GAN

This is a simple GAN with fully connected layers for both generator and discriminator. It is trained on mnist dataset.

### Imports

In [0]:
from keras.models import Model
from keras.layers import Input, Dense, BatchNormalization, Reshape, Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.datasets import mnist
from keras.optimizers import Adam

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image

### Function to build the generator

Generator is a model that takes as input noise and outputs an image.

The noise comes from a normal distribution (0, 1) and is a vector of size `noise_size`.

The generated image is an array of shape `img_shape`.

The presented generator model is a simple network consisted of 3 blocks. Each block has a [Dense](https://keras.io/layers/core/#dense) layer, followed by a [BatchNormalization](https://keras.io/layers/normalization/#batchnormalization) and a [LeakyReLU](https://keras.io/layers/advanced-activations/#leakyrelu).

In [0]:
def build_generator(noise_size, img_shape):
  """
  function that takes as input
  the noise_size (integer) and the img_shape (tuple of integers)
  and returns a keras Model.
  The model has 3 blocks of Dense, BatchNormalization and LeakyReLU layers.
  The units at the Dense layers are 256, 512 and 1024 respectively.
  The alpha parameter at the LeakyReLU layers is 0.2.
  The activation of the last layer is tanh.
  The model (generator) takes as input a tensor of shape (noise_size,)
  and returns a tensor of shape img_shape.
  """
  
  noise = Input((noise_size,))
  
  x = Dense(256)(noise)
  x = BatchNormalization()(x)
  x = LeakyReLU(alpha=0.2)(x)
  
  x = Dense(512)(x)
  x = BatchNormalization()(x)
  x = LeakyReLU(alpha=0.2)(x)
  
  x = Dense(1024)(x)
  x = BatchNormalization()(x)
  x = LeakyReLU(alpha=0.2)(x)
  
  x = Dense(np.prod(img_shape), activation='tanh')(x)
  img = Reshape(img_shape)(x)
  
  generator = Model(noise, img)
  return generator

### Function to build the discriminator

Discriminator is a model that takes as input an image and outputs a number. This number (`validity`) is in [0, 1] and indicates the belief of the discriminator about the validity of the image.
- High value means that the discriminator "thinks" that the given image is a real image:
- Low value means that the discriminator "thinks" that the given image is a generated (fake) one.

The presented discriminator model is a simple network consisted of 3 blocks. Each block has a Dense layer, followed by a LeakyReLU. One could also add a BatchNormalization layer as in the generator case.

In [0]:
def build_discriminator(img_shape):
  """
  function that takes as input the img_shape (tuple of integers)
  and returns a keras Model.
  The model has 3 blocks of Dense and LeakyReLU layers.
  The units at the Dense layers are 1024, 512, and 256 respectively.
  The alpha parameter at the LeakyReLU layers is 0.2
  The activation of the last layer is sigmoid.
  The model (discriminator) takes as input a tensor of shape img_shape
  and returns a tensor of shape 1
  """
  
  img = Input(img_shape)
  f_img = Flatten()(img)
  
  x = Dense(1024)(f_img)
  x = LeakyReLU(alpha=0.2)(x)
  
  x = Dense(512)(x)
  x = LeakyReLU(alpha=0.2)(x)
  
  x = Dense(256)(x)
  x = LeakyReLU(alpha=0.2)(x)
  
  validity = Dense(1, activation='sigmoid')(x)
  
  discriminator = Model(img, validity)
  return discriminator

### Function to compile the models

In keras, one has to compile the models before training them. [Compile](https://keras.io/models/model/) method takes as inputs the optimizer, the loss function and optional metrics.

In our case, we use:
- [Adam](https://keras.io/optimizers/#adam) optimizer
- [binary_crossentropy](https://keras.io/losses/#binary_crossentropy) as a loss function
- (binary) [accuracy](https://keras.io/metrics/#binary_accuracy) as a metric for the discriminator.

We first compile the discriminator model. Then we "freeze" it (make its layers non trainable) and construct a new model. The new model (combined) is the combination of the generator and the discriminator. The combined model takes as input noise, passes it through the generator, takes the generated image and sends it to the discriminator. The disciminator outputs the validity which is the output of the combined model. Finally, we compile the combined model.

In [0]:
def get_compiled_models(generator, discriminator, noise_size):
  """
  function that takes as input
  the generator (keras.Model)
  the discriminator (keras.Model)
  and the noise_size (integer)
  and return the generator, the compiled discriminator and the compiled comnbined models.
  The combined model takes as input noise (tensor of shape (noise_size,))
  and outputs the validity (output of the discriminator)
  of the internally generated image (output of the generator).
  For both models the optimizer is Adam with learning rate 0.0002 and beta_1 0.5
  and the loss function is binary_crossentropy.
  The discriminator has accuracy as metric.
  """
  
  optimizer = Adam(0.0002, 0.5)
  
  discriminator.compile(optimizer, loss='binary_crossentropy', metrics=['accuracy'])
  discriminator.trainable = False
  
  noise = Input((noise_size,))
  img = generator(noise)
  validity = discriminator(img)
  combined = Model(noise, validity)
  
  combined.compile(optimizer, loss='binary_crossentropy')
  
  return generator, discriminator, combined

### Function to sample and save generated images

This is a function that produces some sampled images using the generator and saves them at a folder. This way one can obtain the visual results of the training procedure.

In [0]:
def sample_imgs(generator, noise_size, step):
  """
  function that takes as input the generator (keras.Model), the noise_size (integer)
  and the step (integer) and generates and saves samples of the generated images.
  The images are in a 5x5 grid and are saved at ./images/{step}.png
  """
  
  r, c = 5, 5
  noise = np.random.normal(0, 1, (r*c, noise_size))
  imgs = generator.predict(noise)
  imgs = imgs / 2 + 0.5
  
  fig, axs = plt.subplots(r, c)
  cnt = 0
  for i in range(r):
    for j in range(c):
      axs[i, j].imshow(imgs[cnt], cmap='gray')
      axs[i, j].axis('off')
      cnt += 1
  fig.savefig('./images/%d.png' % step)
  plt.close()

### Function to train the models

This is the function responsible for the training of our GAN. First, we define our real data, in this case we use the [mnist](https://keras.io/datasets/#mnist-database-of-handwritten-digits) dataset. Then we enter the training loop:

- First we train the discriminator for one batch. The discriminator has to be trained at both the real data and the generated data. when training a model, we have to give the target values, so that it can compute the declination from its target (loss) and based on this to become better throuth an optimization method. Since, as we mentioned before, high validity values indicate that the given images are real and low values indicate generated (fake) images, we want the discriminator to ideally output:
 - values close to 1 for the given real images
 - values close to 0 for the given generated images
- After the discriminator we train the combined model. For the combined model we generate images and evaluate their validity through the discriminator. Since we want the generated images to be like the real ones, we want their validity to be ideally close to one. In order to achieve this, we set the target values for the validity of the generated images to be **one**. This is a bit tricky, since when we trained the discriminator, we set the target values for the generated images to zero. The difference is that now we train the generator and thus we want the generated images to be as close to the real ones as possible. And the only way to pass it to the model is by setting the target values for the validity to be the same as those of the real images. However, we don't want the disciminator to "think" that the generated images are indeed real. This is why we "froze" the discriminator at the combined model. This way, the disciminator will not be trained to recognize the generated images as real ones, and the generator will be trained to produce images as close as possible to the real ones.

In [0]:
def train(models, noise_size, img_shape, batch_size, steps):
  """
  function that takes as input the models (tuple of generator, discriminator, combined),
  the noise_size, the img_shape, the batch_size and the steps and trains the models
  for this number of steps on batches of size batch_size.
  The training data are from mnist (keras.datasets).
  For preprocessing, the data are normalized in [-1, 1] (original in [0, 255]).
  Every 100 steps the models loss and accuracy is printed and samples are saved.
  """
  generator, discriminator, combined = models
  #get real data
  (X_train, _), (X_val, _) = mnist.load_data()
  mnist_imgs = np.concatenate((X_train, X_val)) / 127.5 - 1
  
  for step in range(1, steps + 1):
    # train discriminator
    inds = np.random.randint(0, mnist_imgs.shape[0], batch_size)
    real_imgs = mnist_imgs[inds]
    real_validity = np.ones(batch_size)
    
    noise = np.random.normal(0, 1, (batch_size, noise_size))
    gen_imgs = generator.predict(noise)
    gen_validity = np.zeros(batch_size)
    
    r_loss = discriminator.train_on_batch(real_imgs, real_validity)
    g_loss = discriminator.train_on_batch(gen_imgs, gen_validity)
    disc_loss = np.add(r_loss, g_loss) / 2
    
    # train generator
    noise = np.random.normal(0, 1, (batch_size, noise_size))
    gen_validity = np.ones(batch_size)
    gen_loss = combined.train_on_batch(noise, gen_validity)
    
    #print progress
    if step % 100 == 0:
      print('step: %d, D_loss: %f, D_accuracy: %.2f%%, G_loss: %f' % (step, disc_loss[0],
                                                                      disc_loss[1] * 100, gen_loss))
    
    # save_samples
    if step % 100 == 0:
      sample_imgs(generator, noise_size, step)

### Define hyperparameters

In [0]:
%rm -r /content/images
%mkdir /content/images
noise_size = 100
img_shape = 28, 28
batch_size = 64
steps = 1000

### Generate the models

In [0]:
generator = build_generator(noise_size, img_shape)
discriminator = build_discriminator(img_shape)
compiled_models = get_compiled_models(generator, discriminator, noise_size)

### Train the models

In [0]:
train(compiled_models, noise_size, img_shape, batch_size, steps)

### Display samples

In [0]:
Image('/content/images/%d.png' % 800)